In [1]:
!pip install sumy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 21.6 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21693 sha256=9014e54ccc59fa60a2ef99d6b7f5e96480c056cc789c51a0cd4aaed5216684de
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=728336717bef5b5eab7a4c6cbde55b33913a3db536d8bb950c16ce3bef0ea7ab
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [2]:
!pip install pyate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 5.0 MB/s eta 0:00:00
  Created wheel for pyate: filename=pyate-0.5.5-py3-none-any.whl size=18268 sha256=5a7a967277f152ef50dad1a2522dd55ce70e1f5f6e4d531d836f3821610d755b
  Stored in directory: /root/.cache/pip/wheels/c7/03/03/02f61c63a425a29ab036a5f56cd0d7548484354963f4aabd01
Successfully built pyate


In [3]:
!pip install rake-nltk
from rake_nltk import Rake

In [22]:
import spacy
from pyate import combo_basic
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim import corpora
from gensim.models import LdaModel
from collections import defaultdict
import pandas as pd
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from collections import defaultdict
from rake_nltk import Rake
from sklearn.metrics import precision_score, recall_score, f1_score

# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
# Load spacy model
nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [42]:
df = pd.read_excel("Assignment.xlsx")
df

,Article
0,"Retailers, the makers of foods marketed for we..."
1,"Move over, Ozempic — there’s a new drug in tow..."
2,Sept 14 (Reuters) - Bristol Myers Squibb (BMY....
3,Austin Wolcott was 18 years old and pretty sur...
4,"Cancer, often referred to as the “emperor of a..."
5,Nov 28 (Reuters) - The U.S. Food and Drug Admi...
6,Nov 21 (Reuters) - BeiGene (6160.HK) said on T...
7,Sept 19 (Reuters) - Drugmaker BeiGene (6160.HK...
8,BRUKINSA is the first and only BTK inhibitor a...
9,Whether you're looking for a quick bite to eat...


In [43]:
def clean_article(article):
    # Tokenize the article
    words = word_tokenize(article)

    # Remove punctuation and convert to lowercase
    words = [word.lower() for word in words if word.isalpha()]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    cleaned_article = ' '.join(words)
    return cleaned_article

# Clean articles and add a new column 'clean_text'
df['clean_text'] = df['Article'].apply(clean_article)

# Display the preprocessed DataFrame
df


,Article,clean_text
0,"Retailers, the makers of foods marketed for we...",retailer maker food marketed weight loss type ...
1,"Move over, Ozempic — there’s a new drug in tow...",move ozempic new drug town eli lilly zepbound ...
2,Sept 14 (Reuters) - Bristol Myers Squibb (BMY....,sept reuters bristol myers squibb said thursda...
3,Austin Wolcott was 18 years old and pretty sur...,austin wolcott year old pretty sure survive ye...
4,"Cancer, often referred to as the “emperor of a...",cancer often referred emperor malady unyieldin...
5,Nov 28 (Reuters) - The U.S. Food and Drug Admi...,nov reuters food drug administration fda said ...
6,Nov 21 (Reuters) - BeiGene (6160.HK) said on T...,nov reuters beigene said tuesday entered agree...
7,Sept 19 (Reuters) - Drugmaker BeiGene (6160.HK...,sept reuters drugmaker beigene said tuesday wo...
8,BRUKINSA is the first and only BTK inhibitor a...,brukinsa first btk inhibitor approved follicul...
9,Whether you're looking for a quick bite to eat...,whether looking quick bite eat experience quic...


In [44]:
sid = SentimentIntensityAnalyzer()

# Function to get the overall sentiment of an article
def get_article_sentiment(article_text):
    scores = sid.polarity_scores(article_text)

    # Determine the overall sentiment based on the compound score
    if scores['compound'] >= 0.05:
        return 'positive'
    elif scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply the sentiment analysis to each article in the DataFrame
df['mood'] = df['clean_text'].apply(get_article_sentiment)
df

,Article,clean_text,mood
0,"Retailers, the makers of foods marketed for we...",retailer maker food marketed weight loss type ...,positive
1,"Move over, Ozempic — there’s a new drug in tow...",move ozempic new drug town eli lilly zepbound ...,negative
2,Sept 14 (Reuters) - Bristol Myers Squibb (BMY....,sept reuters bristol myers squibb said thursda...,negative
3,Austin Wolcott was 18 years old and pretty sur...,austin wolcott year old pretty sure survive ye...,negative
4,"Cancer, often referred to as the “emperor of a...",cancer often referred emperor malady unyieldin...,negative
5,Nov 28 (Reuters) - The U.S. Food and Drug Admi...,nov reuters food drug administration fda said ...,negative
6,Nov 21 (Reuters) - BeiGene (6160.HK) said on T...,nov reuters beigene said tuesday entered agree...,positive
7,Sept 19 (Reuters) - Drugmaker BeiGene (6160.HK...,sept reuters drugmaker beigene said tuesday wo...,negative
8,BRUKINSA is the first and only BTK inhibitor a...,brukinsa first btk inhibitor approved follicul...,positive
9,Whether you're looking for a quick bite to eat...,whether looking quick bite eat experience quic...,positive


In [45]:
def generate_summary(article_text, num_sentences=1):
    # Initialize parser and tokenizer
    parser = PlaintextParser.from_string(article_text, Tokenizer("english"))

    # Initialize LSA Summarizer
    summarizer = LsaSummarizer()

    # Generate summary
    summary = summarizer(parser.document, num_sentences)
    summary_text = ""
    for sentence in summary:
        summary_text += str(sentence) + " "

    return summary_text

# Apply the function to create a summary for each article in the DataFrame
df['summary'] = df['Article'].apply(generate_summary)

df


,Article,clean_text,mood,summary
0,"Retailers, the makers of foods marketed for we...",retailer maker food marketed weight loss type ...,positive,"Walmart’s chief executive, Doug McMillon, told..."
1,"Move over, Ozempic — there’s a new drug in tow...",move ozempic new drug town eli lilly zepbound ...,negative,Side effects of the weekly injectable include ...
2,Sept 14 (Reuters) - Bristol Myers Squibb (BMY....,sept reuters bristol myers squibb said thursda...,negative,"Bristol, which already has two approved cell t..."
3,Austin Wolcott was 18 years old and pretty sur...,austin wolcott year old pretty sure survive ye...,negative,Though these seismic shifts won’t happen overn...
4,"Cancer, often referred to as the “emperor of a...",cancer often referred emperor malady unyieldin...,negative,The financial toxicity associated with cancer ...
5,Nov 28 (Reuters) - The U.S. Food and Drug Admi...,nov reuters food drug administration fda said ...,negative,Patients and clinical trial participants recei...
6,Nov 21 (Reuters) - BeiGene (6160.HK) said on T...,nov reuters beigene said tuesday entered agree...,positive,Nov 21 (Reuters) - BeiGene (6160.HK) said on T...
7,Sept 19 (Reuters) - Drugmaker BeiGene (6160.HK...,sept reuters drugmaker beigene said tuesday wo...,negative,Sept 19 (Reuters) - Drugmaker BeiGene (6160.HK...
8,BRUKINSA is the first and only BTK inhibitor a...,brukinsa first btk inhibitor approved follicul...,positive,"BeiGene, Ltd. BGNE HKEX: 06160, SSE: 688235)))..."
9,Whether you're looking for a quick bite to eat...,whether looking quick bite eat experience quic...,positive,All restaurants and major chains below will be...


In [46]:
def find_connections(articles, num_topics=5, num_words=5):
    # Preprocess the articles
    processed_articles = [clean_article(article).split() for article in articles]

    # Create a dictionary and corpus
    dictionary = corpora.Dictionary(processed_articles)
    corpus = [dictionary.doc2bow(text) for text in processed_articles]

    # Train the LDA model
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

    # Print the topics
    topics = lda_model.print_topics(num_words=num_words)
    for idx, topic in topics:
        print(f"Topic {idx}: {topic}")

find_connections(df['clean_text'].tolist())

Topic 0: 0.014*"say" + 0.011*"cancer" + 0.008*"cell" + 0.008*"therapy" + 0.005*"patient"
Topic 1: 0.010*"brukinsa" + 0.010*"patient" + 0.009*"beigene" + 0.009*"therapy" + 0.009*"said"
Topic 2: 0.022*"taco" + 0.021*"nike" + 0.017*"strava" + 0.013*"club" + 0.011*"run"
Topic 3: 0.022*"nike" + 0.014*"firm" + 0.007*"market" + 0.007*"product" + 0.007*"believe"
Topic 4: 0.027*"account" + 0.011*"well" + 0.011*"fargo" + 0.010*"checking" + 0.009*"opening"


In [47]:
def aspect_analysis(article, max_aspects=1):
    # Initialize RAKE and VADER Sentiment Analyzer
    r = Rake()
    analyzer = SentimentIntensityAnalyzer()

    # Extract aspects using RAKE
    r.extract_keywords_from_text(article)
    aspects = r.get_ranked_phrases()

    # Initialize variables to track the highest sentiment score and corresponding aspect
    max_sentiment_score = -1  # Initialize to a low value
    top_aspect = None

    # Tokenize sentences
    sentences = nltk.sent_tokenize(article)

    # Process each sentence and aspect
    for sentence in sentences:
        for aspect in aspects:
            if aspect in sentence:
                sentiment = analyzer.polarity_scores(sentence)['compound']
                if sentiment > max_sentiment_score:
                    max_sentiment_score = sentiment
                    top_aspect = aspect

    # Return the top aspect (if found)
    return {top_aspect: round(max_sentiment_score, 2)} if top_aspect else {}

# Iterate over each row in the DataFrame and apply aspect analysis function
for index, row in df.iterrows():
    article_text = row['Article']
    aspect_summary = aspect_analysis(article_text, max_aspects=1)
    aspect_string = str(aspect_summary)
    df.at[index, 'aspect'] = aspect_string

# Print the DataFrame with the extracted aspects
df


,Article,clean_text,mood,summary,aspect
0,"Retailers, the makers of foods marketed for we...",retailer maker food marketed weight loss type ...,positive,"Walmart’s chief executive, Doug McMillon, told...",{'attracted interest': 0.83}
1,"Move over, Ozempic — there’s a new drug in tow...",move ozempic new drug town eli lilly zepbound ...,negative,Side effects of the weekly injectable include ...,{'effective chronic weight management drug': 0...
2,Sept 14 (Reuters) - Bristol Myers Squibb (BMY....,sept reuters bristol myers squibb said thursda...,negative,"Bristol, which already has two approved cell t...",{'company recently received regulatory approva...
3,Austin Wolcott was 18 years old and pretty sur...,austin wolcott year old pretty sure survive ye...,negative,Though these seismic shifts won’t happen overn...,{'skepticism turned': 0.83}
4,"Cancer, often referred to as the “emperor of a...",cancer often referred emperor malady unyieldin...,negative,The financial toxicity associated with cancer ...,{'many patients': 0.61}
5,Nov 28 (Reuters) - The U.S. Food and Drug Admi...,nov reuters food drug administration fda said ...,negative,Patients and clinical trial participants recei...,{'overall safety profile': 0.72}
6,Nov 21 (Reuters) - BeiGene (6160.HK) said on T...,nov reuters beigene said tuesday entered agree...,positive,Nov 21 (Reuters) - BeiGene (6160.HK) said on T...,{'additional payments totaling': 0.27}
7,Sept 19 (Reuters) - Drugmaker BeiGene (6160.HK...,sept reuters drugmaker beigene said tuesday wo...,negative,Sept 19 (Reuters) - Drugmaker BeiGene (6160.HK...,{'jointly developing': 0.49}
8,BRUKINSA is the first and only BTK inhibitor a...,brukinsa first btk inhibitor approved follicul...,positive,"BeiGene, Ltd. BGNE HKEX: 06160, SSE: 688235)))...",{'recent quarterly report': 0.98}
9,Whether you're looking for a quick bite to eat...,whether looking quick bite eat experience quic...,positive,All restaurants and major chains below will be...,{'many contributions workers': 0.82}


In [15]:
# Save the updated DataFrame to a new Excel file
df.to_excel('new_updated_assignment.xlsx', index=False)